In [1]:
import tensorflow as tf
from tensorflow.keras import preprocessing

import numpy as np

In [2]:
samples = ['너 오늘 이뻐 보인다',
          '나는 오늘 기분이 더러워',
          '끝내주는데, 좋은 일이 있나봐',
          '나 좋은 일이 생겼어',
          '아 오늘 진짜 짜증나',
          '환상적인데, 정말 좋은거 같아']

targets =[[1], [0], [1], [1], [0], [1]]

In [3]:
tokenizer = preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(samples)
sequences = tokenizer.texts_to_sequences(samples)
input_sequences = np.array(sequences)
labels = np.array(targets)

word_index = tokenizer.word_index

In [4]:
tokenizer.texts_to_sequences(samples)

[[4, 1, 5, 6],
 [7, 1, 8, 9],
 [10, 2, 3, 11],
 [12, 2, 3, 13],
 [14, 1, 15, 16],
 [17, 18, 19, 20]]

In [5]:
np.array(sequences)

array([[ 4,  1,  5,  6],
       [ 7,  1,  8,  9],
       [10,  2,  3, 11],
       [12,  2,  3, 13],
       [14,  1, 15, 16],
       [17, 18, 19, 20]])

In [6]:
np.array(targets)

array([[1],
       [0],
       [1],
       [1],
       [0],
       [1]])

In [7]:
tokenizer.word_index

{'오늘': 1,
 '좋은': 2,
 '일이': 3,
 '너': 4,
 '이뻐': 5,
 '보인다': 6,
 '나는': 7,
 '기분이': 8,
 '더러워': 9,
 '끝내주는데': 10,
 '있나봐': 11,
 '나': 12,
 '생겼어': 13,
 '아': 14,
 '진짜': 15,
 '짜증나': 16,
 '환상적인데': 17,
 '정말': 18,
 '좋은거': 19,
 '같아': 20}

In [8]:
from tensorflow.keras import backend
from tensorflow.keras import layers

In [9]:
batch_size = 2
num_epochs = 100

In [10]:
vocab_size = len(word_index) + 1
emb_size = 128
hidden_dimension = 256
output_dimension = 1

# Keras Sequential

In [11]:
model = tf.keras.Sequential([
layers.Embedding(vocab_size, emb_size, input_length = 4),
layers.Lambda(lambda x: tf.reduce_mean(x, axis = 1)),
layers.Dense(hidden_dimension, activation='relu'),
layers.Dense(output_dimension, activation='sigmoid')])

model.compile(optimizer=tf.keras.optimizers.Adam(0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [12]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 4, 128)            2688      
                                                                 
 lambda (Lambda)             (None, 128)               0         
                                                                 
 dense (Dense)               (None, 256)               33024     
                                                                 
 dense_1 (Dense)             (None, 1)                 257       
                                                                 
Total params: 35,969
Trainable params: 35,969
Non-trainable params: 0
_________________________________________________________________


In [13]:
model.fit(input_sequences,
          labels,
          epochs=num_epochs,
          batch_size=batch_size)

Epoch 1/100
3/3 [==============================] - 0s 4ms/step - loss: 0.6923 - accuracy: 0.5000
Epoch 2/100
3/3 [==============================] - 0s 2ms/step - loss: 0.6725 - accuracy: 1.0000
Epoch 3/100
3/3 [==============================] - 0s 2ms/step - loss: 0.6550 - accuracy: 1.0000
Epoch 4/100
3/3 [==============================] - 0s 2ms/step - loss: 0.6366 - accuracy: 1.0000
Epoch 5/100
3/3 [==============================] - 0s 2ms/step - loss: 0.6166 - accuracy: 1.0000
Epoch 6/100
3/3 [==============================] - 0s 2ms/step - loss: 0.5929 - accuracy: 1.0000
Epoch 7/100
3/3 [==============================] - 0s 2ms/step - loss: 0.5661 - accuracy: 1.0000
Epoch 8/100
3/3 [==============================] - 0s 1ms/step - loss: 0.5359 - accuracy: 1.0000
Epoch 9/100
3/3 [==============================] - 0s 2ms/step - loss: 0.5008 - accuracy: 1.0000
Epoch 10/100
3/3 [==============================] - 0s 2ms/step - loss: 0.4602 - accuracy: 1.0000
Epoch 11/100
3/3 [===========

# Keras Functional API

In [14]:
inputs = layers.Input(shape = (4, ))
# 입력 데이터는 1차원 배열, 배열의 길이는 4

# 임베딩 레이어-임베딩 벡터로 변환
embed_output = layers.Embedding(vocab_size, emb_size)(inputs)

# 풀링 레이어-평균 임베딩 벡터로 풀링
# 풀링-특징 맵의 크기 줄이기, 계산량 감소, 모델 학습의 안정화
pooled_output = tf.reduce_mean(embed_output, axis=1)

# 은닉 레이어
hidden_layer = layers.Dense(hidden_dimension, activation='relu')(pooled_output)

# 출력 레이어
outputs = layers.Dense(output_dimension, activation='sigmoid')(hidden_layer)

In [15]:
model = tf.keras.Model(inputs=inputs, outputs=outputs)

model.compile(optimizer=tf.keras.optimizers.Adam(0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [16]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 4)]               0         
                                                                 
 embedding_1 (Embedding)     (None, 4, 128)            2688      
                                                                 
 tf.math.reduce_mean (TFOpLa  (None, 128)              0         
 mbda)                                                           
                                                                 
 dense_2 (Dense)             (None, 256)               33024     
                                                                 
 dense_3 (Dense)             (None, 1)                 257       
                                                                 
Total params: 35,969
Trainable params: 35,969
Non-trainable params: 0
_________________________________________________________

In [17]:
model.fit(input_sequences,
          labels,
          epochs=num_epochs,
          batch_size=batch_size)

Epoch 1/100
3/3 [==============================] - 0s 2ms/step - loss: 0.6958 - accuracy: 0.3333
Epoch 2/100
3/3 [==============================] - 0s 1ms/step - loss: 0.6775 - accuracy: 0.8333
Epoch 3/100
3/3 [==============================] - 0s 1ms/step - loss: 0.6634 - accuracy: 1.0000
Epoch 4/100
3/3 [==============================] - 0s 2ms/step - loss: 0.6493 - accuracy: 1.0000
Epoch 5/100
3/3 [==============================] - 0s 1ms/step - loss: 0.6324 - accuracy: 1.0000
Epoch 6/100
3/3 [==============================] - 0s 2ms/step - loss: 0.6146 - accuracy: 1.0000
Epoch 7/100
3/3 [==============================] - 0s 2ms/step - loss: 0.5934 - accuracy: 1.0000
Epoch 8/100
3/3 [==============================] - 0s 2ms/step - loss: 0.5671 - accuracy: 1.0000
Epoch 9/100
3/3 [==============================] - 0s 2ms/step - loss: 0.5391 - accuracy: 1.0000
Epoch 10/100
3/3 [==============================] - 0s 2ms/step - loss: 0.5039 - accuracy: 1.0000
Epoch 11/100
3/3 [===========

# Keras Custom Model

In [18]:
class CustomModel(tf.keras.Model):

    def __init__(self, vocab_size, embed_dimension, hidden_dimension, output_dimension):
        super(CustomModel, self).__init__(name='my_model')
        self.embedding = layers.Embedding(vocab_size, embed_dimension)
        self.dense_layer = layers.Dense(hidden_dimension, activation='relu')
        self.output_layer = layers.Dense(output_dimension, activation='sigmoid')

    def call(self, inputs):
        x = self.embedding(inputs)
        x = tf.reduce_mean(x, axis=1)
        x = self.dense_layer(x)
        x = self.output_layer(x)
        
        return x

In [19]:
model = CustomModel(vocab_size = vocab_size,
            embed_dimension=emb_size,
            hidden_dimension=hidden_dimension,
            output_dimension=output_dimension)

model.compile(optimizer=tf.keras.optimizers.Adam(0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.fit(input_sequences, labels, epochs=num_epochs, batch_size=batch_size)

Epoch 1/100
3/3 [==============================] - 0s 2ms/step - loss: 0.6910 - accuracy: 0.5000
Epoch 2/100
3/3 [==============================] - 0s 1ms/step - loss: 0.6720 - accuracy: 1.0000
Epoch 3/100
3/3 [==============================] - 0s 2ms/step - loss: 0.6539 - accuracy: 1.0000
Epoch 4/100
3/3 [==============================] - 0s 2ms/step - loss: 0.6369 - accuracy: 1.0000
Epoch 5/100
3/3 [==============================] - 0s 2ms/step - loss: 0.6175 - accuracy: 1.0000
Epoch 6/100
3/3 [==============================] - 0s 1ms/step - loss: 0.5962 - accuracy: 1.0000
Epoch 7/100
3/3 [==============================] - 0s 1ms/step - loss: 0.5706 - accuracy: 1.0000
Epoch 8/100
3/3 [==============================] - 0s 2ms/step - loss: 0.5409 - accuracy: 1.0000
Epoch 9/100
3/3 [==============================] - 0s 2ms/step - loss: 0.5038 - accuracy: 1.0000
Epoch 10/100
3/3 [==============================] - 0s 1ms/step - loss: 0.4688 - accuracy: 1.0000
Epoch 11/100
3/3 [===========

# Keras Custom Layer

In [20]:
class CustomLayer(layers.Layer):

    def __init__(self, hidden_dimension, output_dimension, **kwargs):
        self.hidden_dimension = hidden_dimension
        self.output_dimension = output_dimension
        super(CustomLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        self.dense_layer1 = layers.Dense(self.hidden_dimension, activation = 'relu')
        self.dense_layer2 = layers.Dense(self.output_dimension)

    def call(self, inputs):
        hidden_output = self.dense_layer1(inputs)
        return self.dense_layer2(hidden_output)

    # Optional
    def get_config(self):
        base_config = super(CustomLayer, self).get_config()
        base_config['hidden_dim'] = self.hidden_dimension
        base_config['output_dim'] = self.output_dim
        return base_config

    @classmethod
    def from_config(cls, config):
        return cls(**config)

In [21]:
model = tf.keras.Sequential([
    layers.Embedding(vocab_size, emb_size, input_length = 4),
    layers.Lambda(lambda x: tf.reduce_mean(x, axis = 1)),
    CustomLayer(hidden_dimension, output_dimension),
    layers.Activation('sigmoid')])

model.compile(optimizer=tf.keras.optimizers.Adam(0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.fit(input_sequences, labels, epochs=num_epochs, batch_size=batch_size)

Epoch 1/100
3/3 [==============================] - 0s 2ms/step - loss: 0.6995 - accuracy: 0.1667
Epoch 2/100
3/3 [==============================] - 0s 2ms/step - loss: 0.6801 - accuracy: 0.8333
Epoch 3/100
3/3 [==============================] - 0s 2ms/step - loss: 0.6623 - accuracy: 1.0000
Epoch 4/100
3/3 [==============================] - 0s 2ms/step - loss: 0.6465 - accuracy: 1.0000
Epoch 5/100
3/3 [==============================] - 0s 2ms/step - loss: 0.6285 - accuracy: 1.0000
Epoch 6/100
3/3 [==============================] - 0s 1ms/step - loss: 0.6079 - accuracy: 1.0000
Epoch 7/100
3/3 [==============================] - 0s 2ms/step - loss: 0.5849 - accuracy: 1.0000
Epoch 8/100
3/3 [==============================] - 0s 2ms/step - loss: 0.5570 - accuracy: 1.0000
Epoch 9/100
3/3 [==============================] - 0s 2ms/step - loss: 0.5247 - accuracy: 1.0000
Epoch 10/100
3/3 [==============================] - 0s 2ms/step - loss: 0.4870 - accuracy: 1.0000
Epoch 11/100
3/3 [===========